In [1]:
import numpy as np
import hail as hl
from hail import methods
import scipy as sp
import pandas as pd
from math import sqrt, pi
from 

## CREATE GENETIC DATA (and clean/process/edit)

In [2]:
# Create genetic data and write to disk
bnm_mt = hl.balding_nichols_model(3, 100, 1000)
bnm_mt.write("balding_nichols_3_100_1000.mt")

Initializing Hail with default parameters...
Running on Apache Spark version 2.4.1
SparkUI available at http://annamiras-air:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.46-bf7b7f7082e1
LOGGING: writing to /Users/annamira/Documents/GitHub/hail/hail/python/hail/linalg/hail-20200714-0936-0.2.46-bf7b7f7082e1.log
2020-07-14 09:37:07 Hail: INFO: balding_nichols_model: generating genotypes for 3 populations, 100 samples, and 1000 variants...
2020-07-14 09:37:18 Hail: INFO: Coerced sorted dataset
2020-07-14 09:37:21 Hail: INFO: wrote matrix table with 1000 rows and 100 columns in 8 partitions to balding_nichols_3_100_1000.mt


In [3]:
# Read first MatrixTable and clean

# entries are now calls: An object that represents an individual’s call at a genomic locus
mt = hl.read_matrix_table("balding_nichols_3_100_1000.mt")

# don't understand meaning of this: returns the count of non-reference alleles from each call
mt = mt.transmute_entries(n_alt = hl.float64(mt.GT.n_alt_alleles())) 

mt.describe()

----------------------------------------
Global fields:
    'bn': struct {
        n_populations: int32, 
        n_samples: int32, 
        n_variants: int32, 
        n_partitions: int32, 
        pop_dist: array<int32>, 
        fst: array<float64>, 
        mixture: bool
    }
----------------------------------------
Column fields:
    'sample_idx': int32
    'pop': int32
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'ancestral_af': float64
    'af': array<float64>
----------------------------------------
Entry fields:
    'n_alt': float64
----------------------------------------
Column key: ['sample_idx']
Row key: ['locus', 'alleles']
----------------------------------------


In [10]:
# Turn MatrixTable into Table

ht = mt.localize_entries("ent", "sample")
ht.describe()

----------------------------------------
Global fields:
    'bn': struct {
        n_populations: int32, 
        n_samples: int32, 
        n_variants: int32, 
        n_partitions: int32, 
        pop_dist: array<int32>, 
        fst: array<float64>, 
        mixture: bool
    } 
    'sample': array<struct {
        sample_idx: int32, 
        pop: int32
    }> 
----------------------------------------
Row fields:
    'locus': locus<GRCh37> 
    'alleles': array<str> 
    'ancestral_af': float64 
    'af': array<float64> 
    'ent': array<struct {
        n_alt: float64
    }> 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


2020-07-15 11:09:26 Hail: WARN: Name collision: field 'sample' already in object dict. 
  This field must be referenced with __getitem__ syntax: obj['sample']


In [12]:
mt.show()
ht.show()

,,,,
,,0,1,2
locus,alleles,n_alt,n_alt,n_alt
locus<GRCh37>,array<str>,float64,float64,float64
1:1,"[""A"",""C""]",1.00e+00,2.00e+00,1.00e+00
1:2,"[""A"",""C""]",2.00e+00,2.00e+00,1.00e+00
1:3,"[""A"",""C""]",0.00e+00,1.00e+00,2.00e+00
1:4,"[""A"",""C""]",2.00e+00,1.00e+00,1.00e+00
1:5,"[""A"",""C""]",0.00e+00,0.00e+00,1.00e+00
1:6,"[""A"",""C""]",0.00e+00,2.00e+00,2.00e+00
1:7,"[""A"",""C""]",0.00e+00,1.00e+00,0.00e+00


,,,,
locus,alleles,ancestral_af,af,ent
locus<GRCh37>,array<str>,float64,array<float64>,array<struct{n_alt: float64}>
1:1,"[""A"",""C""]",5.39e-01,"[6.64e-01,4.49e-01,6.00e-01]","[(1.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(0.00e+00),(0.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(0.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(0.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(2.00e+00),(0.00e+00),(1.00e+00),(1.00e+00),(0.00e+00),(0.00e+00),(1.00e+00),(1.00e+00),(0.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(0.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(0.00e+00),(1.00e+00),(0.00e+00),(0.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(0.00e+00),(2.00e+00),(1.00e+00),(0.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(0.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(0.00e+00)]"
1:2,"[""A"",""C""]",6.72e-01,"[7.97e-01,5.58e-01,6.04e-01]","[(2.00e+00),(2.00e+00),(1.00e+00),(0.00e+00),(2.00e+00),(2.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(2.00e+00),(0.00e+00),(0.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(0.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(0.00e+00),(2.00e+00),(2.00e+00),(0.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(0.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(0.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(0.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(0.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(0.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(1.00e+00)]"
1:3,"[""A"",""C""]",5.82e-01,"[5.47e-01,5.82e-01,5.60e-01]","[(0.00e+00),(1.00e+00),(2.00e+00),(0.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(0.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(0.00e+00),(1.00e+00),(1.00e+00),(0.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(0.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(0.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(0.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(0.00e+00),(1.00e+00),(2.00e+00),(0.00e+00),(1.00e+00),(1.00e+00),(0.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(0.00e+00),(0.00e+00),(0.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(0.00e+00)]"
1:4,"[""A"",""C""]",5.36e-01,"[3.40e-01,6.99e-01,4.97e-01]","[(2.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(0.00e+00),(0.00e+00),(2.00e+00),(1.00e+00),(0.00e+00),(0.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(0.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(1.00e+00),(1.00e+00),(2.00e+00),(2.00e+00),(2.00e+00),(1.00e+00),(2.00e+00),(2.00e+00)

## Tim's Grouping Methods

In [16]:
# Functions for operating with Tables of ndarrays in Hail (from Tim)

from hail.expr import Expression, ExpressionException, \
    expr_float64, expr_call, expr_any, expr_numeric, expr_array, \
    expr_locus, \
    analyze, check_entry_indexed, check_row_indexed, \
    matrix_table_source, table_source

# Only groups by rows, NOT COLUMNS
def matrix_table_to_table_of_ndarrays(field, group_size, tmp_path = '/tmp/nd_table.ht'):
    """

    The returned table has two fields: 'row_group_number' and 'ndarray'.

    Examples
    --------
    >>> ht = matrix_table_to_table_of_ndarrays(mt.GT.n_alt_alleles(), 100)

    Parameters
    ----------
    field
    group_size
    tmp_path

    Returns
    -------

    """
    mt = matrix_table_source('matrix_table_to_table_of_ndarrays/x', field)
    mt = mt.select_entries(x = field)
    ht = mt.localize_entries(entries_array_field_name='entries')
    # now ht.entries is an array of structs with one field, x

    # we'll also want to mean-impute/variance-normalize/etc here
    ht = ht.select(xs = ht.entries.map(lambda e: e['x']))
    # now ht.xs is an array of float64

    # now need to produce groups of G
    ht = ht.add_index()
    ht = ht.group_by(row_group_number=ht.idx // group_size) \
        .aggregate(ndarray=hl.nd.array(hl.agg.collect(ht.xs)))
    # may require a .T on ndarray

    return ht.checkpoint(tmp_path, overwrite=True)

def chunk_ndarray(a, group_size):
    """Chunks a NDarray along the first axis in chunks of `group_size`.
    Parameters
    ----------
    a
    group_size

    Returns
    -------

    """
    n_groups = a.shape[0] // group_size
    groups = []
    for i in range(a.shape[0] // group_size):
        start = i * group_size
        end = (i + 1) * group_size
        groups.append(a[start:end, :])
    return groups

In [34]:
new_ht = matrix_table_to_table_of_ndarrays(mt.n_alt, 10)
new_ht.show()
mt.show()

2020-07-16 12:57:52 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-07-16 12:57:53 Hail: INFO: wrote table with 100 rows in 8 partitions to /tmp/nd_table.ht


+------------------+
| row_group_number |
+------------------+
|            int64 |
+------------------+
|                0 |
|                1 |
|                2 |
|                3 |
|                4 |
|                5 |
|                6 |
|                7 |
|                8 |
|                9 |
+------------------+

+------------------------------------------------------------------------------+
| ndarray                                                                      |
+------------------------------------------------------------------------------+
| ndarray<float64, 2>                                                          |
+------------------------------------------------------------------------------+
| ndarray{shape=(10, 100), data=[[1.0, 2.0, 1.0, 1.0, 0.0, 0.0, 2.0, 2.0, 1... |
| ndarray{shape=(10, 100), data=[[1.0, 2.0, 0.0, 2.0, 1.0, 2.0, 0.0, 2.0, 0... |
| ndarray{shape=(10, 100), data=[[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2... |
| ndarray{shape=(10, 100), data=[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0... |
| ndarray{shape=(10, 100), data=[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0... |
| ndarray{shape=(10, 100), data=[[1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1... |
| ndarray{shape=(10, 100), data=[[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 2... |
| ndarray{shape=(10, 100), data=[[2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2... |
| ndarray{shape=(10, 100), data=[[2.0, 0.0, 0.0, 2.0, 1.0, 1.0, 1.0, 0.0, 0... |
| ndarray{shape=(10, 100), data=[[2.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1... |
+------------------------------------------------------------------------------+
showing top 10 rows

,,,,
,,0,1,2
locus,alleles,n_alt,n_alt,n_alt
locus<GRCh37>,array<str>,float64,float64,float64
1:1,"[""A"",""C""]",1.00e+00,2.00e+00,1.00e+00
1:2,"[""A"",""C""]",2.00e+00,2.00e+00,1.00e+00
1:3,"[""A"",""C""]",0.00e+00,1.00e+00,2.00e+00
1:4,"[""A"",""C""]",2.00e+00,1.00e+00,1.00e+00
1:5,"[""A"",""C""]",0.00e+00,0.00e+00,1.00e+00
1:6,"[""A"",""C""]",0.00e+00,2.00e+00,2.00e+00
1:7,"[""A"",""C""]",0.00e+00,1.00e+00,0.00e+00


In [33]:
new_ht.describe()

----------------------------------------
Global fields:
    'bn': struct {
        n_populations: int32, 
        n_samples: int32, 
        n_variants: int32, 
        n_partitions: int32, 
        pop_dist: array<int32>, 
        fst: array<float64>, 
        mixture: bool
    } 
----------------------------------------
Row fields:
    'row_group_number': int64 
    'ndarray': ndarray<float64, 2> 
----------------------------------------
Key: ['row_group_number']
----------------------------------------


## Blanczos Algorithm

In [42]:
def makeData(model_input, group_size):
    hold_mt = hl.balding_nichols_model(*model_input)
    hold_mt.write("balding_nichols_test.mt")
    hold_mt = hl.read_matrix_table("balding_nichols_test.mt")
    hold_mt = hold_mt.transmute_entries(n_alt = hl.float64(hold_mt.GT.n_alt_alleles())) 
    table = hold_mt.localize_entries("ent", "sample")
    return matrix_table_to_table_of_ndarrays(hold_mt.n_alt, 4, tmp_path='/tmp/test_table.ht')
    
data = makeData((3, 100, 1000), 4)

2020-07-16 23:10:55 Hail: INFO: balding_nichols_model: generating genotypes for 3 populations, 100 samples, and 1000 variants...
2020-07-16 23:10:56 Hail: INFO: Coerced sorted dataset
2020-07-16 23:10:57 Hail: INFO: wrote matrix table with 1000 rows and 100 columns in 8 partitions to balding_nichols_test.mt
2020-07-16 23:10:59 Hail: INFO: Ordering unsorted dataset with network shuffle
2020-07-16 23:11:02 Hail: INFO: wrote table with 250 rows in 8 partitions to /tmp/test_table.ht


In [43]:
(m, n) = (100, 1000)
k = 50
l = k + 2
q = 0

In [45]:
G = hl.nd.array(np.random.normal(0, 1, (n,l)))

In [67]:
# WANT TO DO H = data @ G
AG = data.annotate_globals(G = G)
AG = AG.annotate(prod = (AG.ndarray @ AG.G))
AG.describe()
matmul_product = AG.aggregate(hl.agg.array_sum(AG.prod))

#(hl.agg.array_sum([data.ndarray @ G]))

----------------------------------------
Global fields:
    'bn': struct {
        n_populations: int32, 
        n_samples: int32, 
        n_variants: int32, 
        n_partitions: int32, 
        pop_dist: array<int32>, 
        fst: array<float64>, 
        mixture: bool
    } 
    'G': ndarray<float64, 2> 
----------------------------------------
Row fields:
    'row_group_number': int64 
    'ndarray': ndarray<float64, 2> 
    'prod': ndarray<float64, 2> 
----------------------------------------
Key: ['row_group_number']
----------------------------------------


TypeError: array_sum: parameter 'expr': expected expression of type array<int64 or float64>, found <NDArrayNumericExpression of type ndarray<float64, 2>>

## Practice Scraps - linalg operations on small data, experimenting with Hail

In [9]:
# annotate_{rows, cols, entries}
# can annotate without aggregating but can also do an aggregation that is called an annotation?

# mt.annotate_rows(sum_of_ef1_by_row=hl.agg.sum(mt.ef1))
# Aggregate along each row of entries to create a new row annotation. Can
# reference column and entry fields in aggregations.

# mt.annotate_cols(sum_of_ef1_by_col=hl.agg.sum(mt.ef1))
# Aggregate along each column of entries to create a new col annotation.
# Can reference row and entry fields in aggregations

# need map-like aggregator to create "new" MatrixTable that is a product

AttributeError: 'NDArrayNumericExpression' object has no attribute 'typ'

In [21]:
# Practice doing a matrix multiplication and a transpose

# Make some python ndarrays
a = np.arange(30).reshape((5, 6))
b = np.arange(24).reshape((6, 4))

# Make Hail ndarrays
matrix_5_6 = hl.nd.array(a)
matrix_6_4 = hl.nd.array(b)

# Make MatrixTables from pandas dataframes
dfA = pd.DataFrame(data=a[1:,1:], index=a[1:,0], columns=a[0,1:])
dfB = pd.DataFrame(data=b[1:,1:], index=b[1:,0], columns=b[0,1:])

# How do these hail ndarrys carry across into MatrixTable use?

In [22]:
hl.eval(matrix_5_6) 

array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29]])

In [24]:
dfA

,1,2,3,4,5
6,7,8,9,10,11
12,13,14,15,16,17
18,19,20,21,22,23
24,25,26,27,28,29


In [25]:
tableA = hl.Table.from_pandas(dfA)

In [26]:
tableA.show()

,,,,
1,2,3,4,5
int64,int64,int64,int64,int64
7,8,9,10,11
13,14,15,16,17
19,20,21,22,23
25,26,27,28,29


In [35]:
mtA = tableA.to_matrix_table(row_key=['1'], col_key=['2'])

In [36]:
mtA.describe()
new_htA = matrix_table_to_table_of_ndarrays(mtA.1, 2, PATH???)

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    '2': int64
----------------------------------------
Row fields:
    '1': int64
----------------------------------------
Entry fields:
    '3': int64
    '4': int64
    '5': int64
----------------------------------------
Column key: ['2']
Row key: ['1']
----------------------------------------


FatalError: HailException: Expected identifier but found integer '2'.
<input>:1:(TableCount (MatrixColsTable (CastTableToMatrix __uid_34 __uid_33 (`2`) (TableMapGlobals (TableKeyByAndAggregate None 50 (TableMapGlobals (TableKeyBy () False (JavaTable m1)) (InsertFields (SelectFields () (Ref global)) None (__uid_33 (Let __uid_31 (Apply dictToArray () Array[Tuple[Struct{2:Int64},Struct{}]] (LiftMeOut (TableAggregate (TableKeyBy () False (JavaTable m1)) (AggLet __cse_1 False (SelectFields (`1` `2` `3` `4` `5`) (Ref row)) (AggGroupBy False (SelectFields (`2`) (Ref __cse_1)) (Apply indexArray () Struct{} (ApplyAggOp Take ((I32 1)) ((SelectFields () (Ref __cse_1)))) (I32 0) (Str "  File \"<ipython-input-35-f54da47f85f9>\", line 1, in <module>\n    mtA = tableA.to_matrix_table(row_key=['1'], col_key=['2'])\n\n  File \"<decorator-gen-1131>\", line 2, in to_matrix_table\n\n  File \"/Users/annamira/Documents/GitHub/hail/hail/python/hail/table.py\", line 2961, in to_matrix_table\n    hl.agg.group_

In [20]:
tableA.group_by(row_group_number=tableA.idx // 2)

AttributeError: 'NDArrayNumericExpression' object has no attribute 'group_by'